In [1]:
import os
import json
import pathlib
from datetime import datetime
import optuna
import pandas as pd
import numpy as np
from scipy.stats import rankdata
import lightgbm as lgb
import torch
from typing import List, Dict, Union, Tuple, NamedTuple
from tqdm import tqdm
import scml
import mylib

In [2]:
em_enable = False
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
job_dir = f"models/lgb/{ts}"
pathlib.Path(job_dir).mkdir(parents=True, exist_ok=True)
char_fs = ["length", "digit_frac", "letter_frac", "space_frac", "punc_frac", "upper_frac", 
           "repeat_char_frac", "repeat_substring_frac"]
textstat_fs = ["syllables_per_word", "syllables_per_sent", "words_per_sent", "flesch_reading_ease", 
           "flesch_kincaid_grade", "gunning_fog", "smog_index", "automated_readability_index", 
           "coleman_liau_index", "linsear_write_formula", "dale_chall_readability_score"]
dtfy_fs = ['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 
           'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 
           'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 
           'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']
hatebert_fs = ["hb_bert_off", "hb_bert_abu", "hb_hatebert_off", "hb_hatebert_abu"]
tweeteval_fs = ["te_roberta_off", "te_roberta_emo_anger", "te_roberta_snt_neg", 
                "te_roberta_iro", "te_xlm_roberta_snt_neg"]
vad_fs = ["valence_min", "valence_max", "valence_mean", "arousal_min", "arousal_max", "arousal_mean"]
vocabulary = []
with open("data/vocab.json") as f:
    tmp = json.load(f)
    vocabulary = tmp["term"]
print(f"len(vocab)={len(vocabulary)}")
ti_fs = []
for t in vocabulary:
    words = ["ti"] + t.split()
    ti_fs.append("_".join(words))
features = char_fs + textstat_fs + dtfy_fs + hatebert_fs + tweeteval_fs + ti_fs + vad_fs
val_features = list(features)
if em_enable:
    em_size = 384
    em_cols = [f"paraphrase-MiniLM-L6-v2_{i:04d}" for i in range(em_size)]
    features += em_cols
    em_cols = [f"zz{i:04d}" for i in range(em_size)]
    val_features += em_cols
features.sort()
val_features.sort()
print(f"{len(features)} features")

len(vocab)=4098
4152 features


In [3]:
class Conf(NamedTuple):
    job_dir: str = job_dir
    device: torch.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_boost_round: int = 4000
    lr: Tuple[float, float] = (1e-3, 1e-3)
    feature_fraction: Tuple[float, float] = (1, 1)
    min_data_in_leaf: Tuple[int, int] = (20, 20)
    label: str = "label"
    query: str = "worker"
    objective: str = "lambdarank"
    n_trials: int = 1
    features: List[str] = features
    val_features: List[str] = val_features
        

conf = Conf()
print(conf)
if conf.device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Conf(job_dir='models/lgb/20220207_175155', device=device(type='cuda'), num_boost_round=4000, lr=(0.001, 0.001), feature_fraction=(1, 1), min_data_in_leaf=(20, 20), label='label', query='worker', objective='lambdarank', n_trials=1, features=['arousal_max', 'arousal_mean', 'arousal_min', 'automated_readability_index', 'coleman_liau_index', 'dale_chall_readability_score', 'digit_frac', 'dtm_identity_attack', 'dtm_insult', 'dtm_obscene', 'dtm_severe_toxicity', 'dtm_sexual_explicit', 'dtm_threat', 'dtm_toxicity', 'dto_identity_attack', 'dto_insult', 'dto_obscene', 'dto_severe_toxicity', 'dto_threat', 'dto_toxicity', 'dtu_identity_attack', 'dtu_insult', 'dtu_obscene', 'dtu_severe_toxicity', 'dtu_sexual_explicit', 'dtu_threat', 'dtu_toxicity', 'flesch_kincaid_grade', 'flesch_reading_ease', 'gunning_fog', 'hb_bert_abu', 'hb_bert_off', 'hb_hatebert_abu', 'hb_hatebert_off', 'length', 'letter_frac', 'linsear_write_formula', 'punc_frac', 'repeat_char_frac', 'repeat_substring_frac', 'smog_index', '

In [4]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [5]:
train = pd.read_parquet("input/tra.parquet")
# Sort by query groups
train.sort_values("worker", inplace=True, ignore_index=True)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191828 entries, 0 to 191827
Data columns (total 4538 columns):
 #     Column                        Non-Null Count   Dtype  
---    ------                        --------------   -----  
 0     bws                           191828 non-null  float32
 1     worker                        191828 non-null  int8   
 2     length                        191828 non-null  int16  
 3     digit_frac                    191828 non-null  float32
 4     letter_frac                   191828 non-null  float32
 5     space_frac                    191828 non-null  float32
 6     punc_frac                     191828 non-null  float32
 7     upper_frac                    191828 non-null  float32
 8     repeat_char_frac              191828 non-null  float32
 9     repeat_substring_frac         191828 non-null  float32
 10    syllables_per_word            191828 non-null  float32
 11    syllables_per_sent            191828 non-null  float32
 12    words_per_se

# Label Gain

In [6]:
# Force unique ranks
col = conf.label
train[col] = rankdata(train["bws"], method='ordinal')
train[col] = train[col].astype(np.int32)
cols = [conf.label, "bws"]
train[cols].describe(percentiles=percentiles)

,label,bws
count,191828.000000,191828.000000
mean,95914.500000,-0.025554
std,55376.118056,0.137211
min,1.000000,-0.889000
1%,1919.270000,-0.319940
5%,9592.350000,-0.221003
10%,19183.700000,-0.175615
20%,38366.400000,-0.125782
30%,57549.100000,-0.092675
40%,76731.800000,-0.065431


In [7]:
val = pd.read_parquet("input/val.parquet")
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 4537 columns):
 #     Column                        Non-Null Count  Dtype  
---    ------                        --------------  -----  
 0     text                          14251 non-null  object 
 1     length                        14251 non-null  int16  
 2     digit_frac                    14251 non-null  float32
 3     letter_frac                   14251 non-null  float32
 4     space_frac                    14251 non-null  float32
 5     punc_frac                     14251 non-null  float32
 6     upper_frac                    14251 non-null  float32
 7     repeat_char_frac              14251 non-null  float32
 8     repeat_substring_frac         14251 non-null  float32
 9     syllables_per_word            14251 non-null  float32
 10    syllables_per_sent            14251 non-null  float32
 11    words_per_sent                14251 non-null  float32
 12    flesch_reading_ease         

In [8]:
vd = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
vd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB


In [9]:
def group_sizes(groups: List[int]) -> List[int]:
    """Groups must be a sorted list."""
    n = 1
    res = []
    for i in range(1, len(groups)):
        if groups[i] != groups[i-1]:
            res.append(n)
            n = 0
        n += 1
    res.append(n)
    return res

In [10]:
class LgbObjective:
    def __init__(
        self,
        train,
        val,
        validation_data,
        conf: Conf,
    ):
        self.conf = conf
        x_train = train[self.conf.features].to_numpy()
        y_train = train[self.conf.label].to_numpy()
        self.label_gain = list(train[self.conf.label])
        self.label_gain.sort()
        self.label_gain.append(len(train) + 1)
        group = group_sizes(train[self.conf.query])
        self.ds = lgb.Dataset(x_train, label=y_train, group=group)
        self.val_texts = list(val["text"])
        self.x_val = val[self.conf.val_features].to_numpy()
        self.validation_data = validation_data
        self.history: List[Dict[str, Union[str, int, float]]] = []

    def __call__(self, trial):
        hist = {
            "trial_id": trial.number,
            "feature_fraction": trial.suggest_uniform(
                "feature_fraction", self.conf.feature_fraction[0], self.conf.feature_fraction[1]
            ),
            "lr": trial.suggest_loguniform(
                "lr", self.conf.lr[0], self.conf.lr[1]
            ),
            "min_data_in_leaf": trial.suggest_int(
                "min_data_in_leaf", self.conf.min_data_in_leaf[0], self.conf.min_data_in_leaf[1]
            ),
        }
        b = lgb.train(
            {
                'objective': self.conf.objective,
                #'lambda_l1': 1,
                'feature_fraction': hist['feature_fraction'],
                'learning_rate': hist['lr'],
                "min_data_in_leaf": hist["min_data_in_leaf"],
                "label_gain": self.label_gain,
                "force_col_wise": True,
                "verbose": 1,
            },
            self.ds,
            num_boost_round=conf.num_boost_round,
        )
        directory = f"{self.conf.job_dir}/trial_{hist['trial_id']}"
        pathlib.Path(directory).mkdir(parents=True, exist_ok=True)
        b.save_model(f"{directory}/model.txt")
        y_pred = b.predict(self.x_val)
        y_pred = rankdata(y_pred, method="ordinal")
        preds = {}
        for i in range(len(y_pred)):
            preds[self.val_texts[i]] = y_pred[i] 
        hist["score"] = mylib.comp_metric(preds, validation_data=self.validation_data)
        self.history.append(hist)
        return hist["score"]

In [11]:
obj = LgbObjective(
    train=train,
    val=val,
    validation_data=vd,
    conf=conf,
)
study = optuna.create_study(direction="maximize")
study.optimize(obj, n_trials=conf.n_trials)

[I 2022-02-07 17:52:09,429] A new study created in memory with name: no-name-c439976d-1094-497c-9c06-d0290156ef87


[LightGBM] [Info] Total Bins 94625
[LightGBM] [Info] Number of data points in the train set: 191828, number of used features: 1501


[I 2022-02-07 17:54:50,507] Trial 0 finished with value: 0.6968247641822771 and parameters: {'feature_fraction': 1.0, 'lr': 0.001, 'min_data_in_leaf': 20}. Best is trial 0 with value: 0.6968247641822771.


In [12]:
df = pd.DataFrame.from_records(obj.history)
df.sort_values("score", ascending=False, inplace=True, ignore_index=True)
_path = f"{job_dir}/cv.csv"
df.to_csv(_path, index=False)
print(f"Saved {_path}")
df.head(conf.n_trials)

Saved models/lgb/20220207_175155/cv.csv


,trial_id,feature_fraction,lr,min_data_in_leaf,score
0,0,1.0,0.001,20,0.696825


In [13]:
x_train = train[conf.features].to_numpy()
y_train = train[conf.label].to_numpy()
label_gain = list(train[conf.label])
label_gain.sort()
label_gain.append(len(train) + 1)
print(f"label_gain min={min(label_gain)}, max={max(label_gain)}")
group = group_sizes(train[conf.query])
print(f"group sizes={group}")

label_gain min=1, max=191829
group sizes=[5710, 7632, 7883, 7707, 7724, 7727, 7879, 7849, 7892, 7622, 7663, 7733, 7803, 7850, 7690, 7831, 7822, 7709, 7700, 7659, 7669, 7812, 7704, 7855, 7703]


In [14]:
%%time
best = df.iloc[0]
b = lgb.train(
    {
        'objective': conf.objective,
        #'lambda_l1': 1,
        'feature_fraction': best['feature_fraction'],
        'learning_rate': best['lr'],
        "min_data_in_leaf": int(best["min_data_in_leaf"]),
        "label_gain": label_gain,
        "force_col_wise": True,
        "verbose": 1,
    },
    lgb.Dataset(x_train, label=y_train, group=group),
    num_boost_round=conf.num_boost_round,
)
_path = f"{job_dir}/model.txt"
b.save_model(_path)
print(f"Saved {_path}")

[LightGBM] [Info] Total Bins 94625
[LightGBM] [Info] Number of data points in the train set: 191828, number of used features: 1501
Saved models/lgb/20220207_175155/model.txt
Wall time: 2min 45s


In [15]:
%%time
scores = b.feature_importance()
assert len(scores) == len(features)
rows = []
for i, score in enumerate(scores):
    row = {'importance': score, 'feature': features[i]}
    rows.append(row)
df = pd.DataFrame.from_records(rows)
df.sort_values('importance', ascending=False, inplace=True, ignore_index=True)
_path = f"{job_dir}/features.csv"
df.to_csv(_path, index=True)
print(f"Saved {_path}")
df.T.head()

Saved models/lgb/20220207_175155/features.csv
Wall time: 92 ms


0                    1                2                   3     \
importance    6139                 6059             5700                5090   
feature     length  dtm_sexual_explicit  hb_hatebert_abu  te_roberta_snt_neg   

                   4            5               6                    7     \
importance         4204         4111            3768                 3524   
feature     letter_frac  dto_obscene  te_roberta_off  dtu_sexual_explicit   

                           8                    9             10    \
importance                 3482                 3165          3059   
feature     dtu_severe_toxicity  dto_severe_toxicity  dtm_toxicity   

                       11          12                13    \
importance             2882        2523              2451   
feature     hb_hatebert_off  dtu_insult  repeat_char_frac   

                                    14         15            16    \
importance                          2407       2270          2183   
feature     dale_chall_readability_score  punc_frac  valence_mean   

                           17      18          19                  20    \
importance                 2134    2041        2011                1999   
feature     dtm_severe_toxicity  ti_ass  upper_frac  syllables_per_word   

                    21          22       23              24    \
importance          1989        1795     1744            1733   
feature     dto_toxicity  dtm_insult  ti_fuck  te_roberta_iro   

                           25          26                   27    \
importance                 1498        1464                 1422   
feature     dtu_identity_attack  digit_frac  dtm_identity_attack   

                           28           29          30          31    \
importance                 1421         1370        1318        1267   
feature     flesch_reading_ease  hb_bert_abu  space_frac  dto_insult   

                   32                    33           34    \
importance         1144                  1142         1139   
feature     arousal_max  te_roberta_emo_anger  arousal_min   

                                   35            36       37          38    \
importance                         1120          1119     1093        1088   
feature     automated_readability_index  arousal_mean  ti_cock  dtu_threat   

                           39                      40              41    \
importance                 1041                    1020             966   
feature     dto_identity_attack  te_xlm_roberta_snt_neg  words_per_sent   

                   42          43          44         45    \
importance          951         933         927        923   
feature     hb_bert_off  smog_index  dto_threat  ti_faggot   

                             46           47      48           49    \
importance                    906          902     894          876   
feature     linsear_write_formula  dtu_obscene  ti_die  gunning_fog   

                  50                  51      52         53           54    \
importance         826                 819     780        761          744   
feature     dtm_threat  coleman_liau_index  ti_gay  ti_fuckin  dtm_obscene   

                  55       56           57            58    \
importance         714      673          661           640   
feature     ti_fucking  ti_slur  valence_max  dtu_toxicity   

                          59                    60           61         62    \
importance                 613                   564          446        357   
feature     syllables_per_sent  flesch_kincaid_grade  valence_min  ti_vagina   

               63               64          65       66            67    \
importance      310              303         279      277           265   
feature     ti_suck  ti_motherfucker  ti_asshole  ti_cunt  ti_profanity   

               68       69        70                     71         72    \
importance      258      233       226                    195        163   
feat